In [2]:
# pip install whyshift
# pip install folktables
import pandas as pd
import numpy as np
import cvxpy as cp

In [26]:
import seaborn
print(seaborn.__version__)

0.12.2


In [3]:
state_list = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI',
              'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI',
              'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC',
              'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT',
              'VT', 'VA', 'WA', 'WV', 'WI', 'WY', 'PR']

In [4]:
_STATE_CODES = {'AL': 1, 'AK': 2, 'AZ': 4, 'AR': 5, 'CA': 6,
                'CO': 8, 'CT': 9, 'DE': 10, 'FL': 12, 'GA': 13,
                'HI': 15, 'ID': 16, 'IL': 17, 'IN': 18, 'IA': 19,
                'KS': 20, 'KY': 21, 'LA': 22, 'ME': 23, 'MD': 24,
                'MA': 25, 'MI': 26, 'MN': 27, 'MS': 28, 'MO': 29,
                'MT': 30, 'NE': 31, 'NV': 32, 'NH': 33, 'NJ': 34,
                'NM': 35, 'NY': 36, 'NC': 37, 'ND': 38, 'OH': 39,
                'OK': 40, 'OR': 41, 'PA': 42, 'RI': 44, 'SC': 45,
                'SD': 46, 'TN': 47, 'TX': 48, 'UT': 49, 'VT': 50,
                'VA': 51, 'WA': 53, 'WV': 54, 'WI': 55, 'WY': 56,
                'PR': 72}

In [5]:
INV_STATE_CODES = {v: k for k, v in _STATE_CODES.items()}

In [6]:
STATES = state_list

# Data Loading and Preprocessing

In [7]:
from folktables import ACSDataSource, ACSEmployment

data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')
acs_data = data_source.get_data(states=STATES, download=True)
features, label, group = ACSEmployment.df_to_numpy(acs_data)

In [8]:
def adult_filter(data):
    """Mimic the filters in place for Adult data.

    Adult documentation notes: Extraction was done by Barry Becker from
    the 1994 Census database. A set of reasonably clean records was extracted
    using the following conditions:
    ((AAGE>16) && (AGI>100) && (AFNLWGT>1)&& (HRSWK>0))
    """
    df = data
    df = df[df['AGEP'] > 16]
    df = df[df['PINCP'] > 100]
    df = df[df['WKHP'] > 0]
    df = df[df['PWGTP'] >= 1]
    return df

In [9]:
# Selected features from whyshift
features=[
        'AGEP',     # age
        'COW',      # class of workers
        'SCHL',     # education attainments
        'MAR',      # marriage status
        'OCCP',     # occupation code
        'RELP',     # relationship code
        'WKHP',     # usual hours worked per week
        'SEX',      # sex code
        'RAC1P',    # race code
    ]

In [10]:
target='PINCP'
#target_transform = lambda x: x > 50000

In [11]:
postprocess = lambda x: np.nan_to_num(x, -1)

In [12]:
def df_to_numpy(data):

    df = adult_filter(data)

    res = []
    for feature in features:
        res.append(df[feature].to_numpy())
    res_array = np.column_stack(res)

    y = df[target].to_numpy()
    group = df['ST'].to_numpy()

    return postprocess(res_array), y, group

In [13]:
def df_to_pandas(data):
   
    df = adult_filter(data)
    variables = df[features]
    variables = pd.DataFrame(postprocess(variables.to_numpy()),
                             columns=variables.columns)
    variables['LogIncome'] = pd.DataFrame(np.log(df[target])).reset_index(drop=True)

    y = pd.DataFrame(df[target]).reset_index(drop=True)
    #pd.DataFrame(target_transform(df[target])).reset_index(drop=True)
    #group = pd.DataFrame(df['ST']).reset_index(drop=True)
    group = df['ST'].to_numpy()
    group = [INV_STATE_CODES[group[i]] for i in range(len(group))]

    return variables, y, group

In [14]:
#X, y, group = df_to_numpy(acs_data)
X, y, group = df_to_pandas(acs_data)

In [15]:
new_df = X
new_df.insert(loc = len(new_df.columns), column = "Y", value = y)
new_df.insert(loc = len(new_df.columns), column = "STATE", value = group)

In [16]:
new_df

,AGEP,COW,SCHL,MAR,OCCP,RELP,WKHP,SEX,RAC1P,LogIncome,Y,STATE
0,18.0,1.0,18.0,5.0,4720.0,17.0,21.0,2.0,2.0,7.377759,1600.0,AL
1,53.0,5.0,17.0,5.0,3605.0,16.0,40.0,1.0,1.0,9.210340,10000.0,AL
2,41.0,1.0,16.0,5.0,7330.0,17.0,40.0,1.0,1.0,10.085809,24000.0,AL
3,18.0,6.0,18.0,5.0,2722.0,17.0,2.0,2.0,1.0,5.192957,180.0,AL
4,21.0,5.0,19.0,5.0,3870.0,17.0,50.0,1.0,1.0,10.275051,29000.0,AL
...,...,...,...,...,...,...,...,...,...,...,...,...
1664495,39.0,6.0,16.0,5.0,6260.0,0.0,20.0,1.0,1.0,9.169518,9600.0,PR
1664496,38.0,6.0,14.0,5.0,4251.0,0.0,32.0,1.0,8.0,7.783224,2400.0,PR
1664497,37.0,1.0,19.0,3.0,7750.0,13.0,40.0,2.0,9.0,9.888374,19700.0,PR
1664498,47.0,1.0,16.0,1.0,8990.0,1.0,40.0,1.0,8.0,9.836279,18700.0,PR


In [17]:
new_df.to_csv("preprocessed_income_2018.csv")